# MNIST Dataset Digit Recognition using a Neural Network
### Isaac Mattern, Jamie Walters, Noah LaPolt, Sam Gernstetter
CSE 3521 - Intro AI at The Ohio State Univeristy

## Load Data

In [5]:
import numpy as np
import pandas as pd

# Import data: this data is from https://www.kaggle.com/c/digit-recognizer/data
# Download, then upload to this colab project
train = pd.read_csv("train.csv")
# Test csv has no labels column. Submission is a csv, Kaggle will tell us our accuracy. 
x_test = pd.read_csv("test.csv")

In [6]:
# Separate training dataframe into labels (the actual digit) and the data we are looking at
y_train = train["label"]
x_train = train.drop(columns=["label"])

del train

## Normalize the data
Currently, there is a value from 0 to 255 to indicate the darkness of the pixel. We will divide this value by 255 to get a decimal value between 0 and 1.

In [7]:
x_train = x_train / 255.0
x_test = x_test / 255.0

## Neural Network Function

In [8]:
# Sigmoid function to keep values small enough.
def sigmoid(a):
    return 1/(1 + np.exp(-a))

In [9]:
# Calculation for the loss based on correct answers.
def loss(w, y, x):
  return 2 * (sigmoid(np.matmul(w, x)) - y)

In [12]:
# The neural network training algorithm.
def nn(x, y, learningRate=0.01, max_iter=10):
  # Starts weight arrays with random numbers.
  w1 = np.random.randint(-1, high=2, size=(98, x.shape[1])) # Hiden layer weights.
  w2 = np.random.randint(-1, high=2, size=(10, 98)) # Output layer weights.
  s = x.shape[0]                 # Number of test variables.
  x = x.values                   # X as a list.
  layer = 8                      # Number of layers in each group pass.

  # Repeats training for higher accuracy to.
  for t in range(max_iter):
    print_num = np.random.randint(int(s / layer)) # The guess printed.

    # Loop through all training images.
    for i in range(int(s / layer)):
      current_w1 = []
      current_w2 = []

      # Groups images to save on ram and increase speed.
      for k in range(layer):
        current = int(i * layer) + k  # The current image index.
        h2_label = np.zeros((1, 10))  # The correct label for data.
        h2_label[0][y[current]] = 1
        
        # Values of the hidden layer.
        h1 = sigmoid(np.matmul(w1, x[current]))

        # Find loss using chain rule.
        h2_loss = loss(w2, h2_label, h1)  # Loss of the output.
        h1_loss = np.matmul(h2_loss, w2)  # Loss of the hidden layer.

        # Loss of the weights for each layer.
        w2_loss = np.matmul(np.array([h1]).transpose(), h2_loss).transpose()
        w1_loss = np.matmul(np.array([x[current]]).transpose(), h1_loss).transpose()

        # Store image loss for each layer.
        current_w2.append(w2_loss)
        current_w1.append(w1_loss)

      # Update the loss based on the sum of loss of the layer.
      w2 = np.add(w2, -learningRate * (sum(current_w2) / layer))
      w1 = np.add(w1, -learningRate * (sum(current_w1) / layer))

      # Prints the current iteration and the computers current guess.
      if (i == print_num): 
        guess = sigmoid(np.matmul(w2, h1))
        print("Iteration", t + 1)
        print("Guess:", np.where(guess == max(guess))[0][0], "Was:", y[current])
        print("--------------")
  return (w1, w2)

In [13]:
n = nn(x_train, y_train, max_iter=4)

Iteration 1
Guess: 2 Was: 2
--------------
Iteration 2
Guess: 6 Was: 6
--------------
Iteration 3
Guess: 5 Was: 3
--------------
Iteration 4
Guess: 1 Was: 1
--------------


## Testing

In [14]:
def test(x, n):
  data = []     # The final guesses as a list
  x = x.values  # X as a list.

  # Loop through all test images.
  for i in range(x.shape[0]):
    # Multiple the pixel values by the weights to get a 10x1 matrix of answers.
    guess = sigmoid(np.matmul(n[1], sigmoid(np.matmul(n[0], x[i]))))
    # guess = np.matmul(n[1], x[i])
    # Append the index of the highest value to the data.
    data.append([i + 1, np.where(guess == max(guess))[0][0]])

  # Returns data as data frame.
  return pd.DataFrame(data, columns =['ImageId', 'Label'])

In [15]:
data = test(x_train, n)
correct = 0
total = 0

for i, row in enumerate(data.iterrows()):
  val = row[1].values
  if val[1] == y_train[i]:
    correct += 1
  # else:
  #   print("Guessed:", val[1], "Was:", y_train[i])
  total += 1

print("Train accuracy:", correct / total)


Train accuracy: 0.9055238095238095


In [ ]:
data = test(x_test, n)

# Writes the file into a CSV for checking.
with open('results.csv', 'w') as writefile:
  writefile.write("ImageId,Label\n")
  for row in data.iterrows():
    val = row[1].values
    writefile.write(str(val[0]))
    writefile.write(",")
    writefile.write(str(val[1]))
    writefile.write("\n")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
